In [7]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
from plotnine import ggplot, aes, geom_line, facet_wrap, labs
import plotly.express as px
%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

import matplotlib.pyplot as plt


#Dash
import dash
import dash_core_components as dcc
from dash import html
from dash.dependencies import Input, Output
import re

from datetime import datetime, timedelta


# Cleaning to get Net Riders



In [3]:
file_path = "/Users/jeremyrodriguez/Desktop/PMC Data/PMC Project/TrackingMain.csv"

df= pd.read_csv(file_path)

df=df.drop('Unnamed: 0',axis = 1)

df['Direction'] = df['Location In'].apply(lambda x: x.split(' ')[1].lower() if len(x.split(' ')) > 1 else 'unknown')

# Replace the second word with 'in' or 'out' if it matches, otherwise leave it as is
df['Direction'] = df['Direction'].apply(lambda x: 'in' if 'in' in x else ('out' if 'out' in x else x))

df['Location'] = df['Location In'].str.replace(r' in| out', '', case=False, regex=True)






In [9]:
rows = []

# Looping over unique eGIFTIDs and Locations
for (egift_id, location), group in df.groupby(['eGIFTID', 'Location']):
    time_in = time_out = None
    
    # If there is a 'in' Direction entry, assign the corresponding 'In time' value to 'Time In'
    if 'in' in group['Direction'].values:
        time_in = group.loc[group['Direction'] == 'in', 'time'].iloc[0]
    
    # If there is a 'out' Direction entry, assign the corresponding 'In time' value to 'Time Out'
    if 'out' in group['Direction'].values:
        time_out = group.loc[group['Direction'] == 'out', 'time'].iloc[0]
    
    # Append the consolidated row to the list
    rows.append({
        'eGIFTID': egift_id,
        'Location': location,
        'Time In': time_in,
        'Time Out': time_out
    })

# Convert the list of rows to a DataFrame
clean_df= pd.DataFrame(rows)

clean_df
clean_df.to_csv('clean_tracking.csv', index = False)

Using Clean DF for graph

ValueError: time data 'None' does not match format '%H:%M:%S'

In [ ]:



# #Pull Location and Direction. Will be useful for having the total volume over time
# df['Location'] = df['Location In'].str.replace(r' in| out', '', case=False, regex=True)
# df=df.drop('Unnamed: 0',axis = 1)
# df=df.drop("Hour", axis =1)

# # Creating a new 'Direction' column with values 'in' or 'out'
# df['Direction'] = df['Location In'].apply(lambda x: 'in' if 'in' in x.lower() else 'out')
# df['Net Riders'] = df['Location In'].apply(lambda x: 1 if 'in' in [word.lower() for word in x.split()] else -1)


# # Calculating the Cumulative Sum of Net Riders
# df['Total Net Riders'] = df.groupby('Location')['Net Riders'].cumsum()



# df.sort_values(['Location', 'eGIFTID'], inplace=True)

# # Keep only the first two entries for each 'Gift ID' for each 'Location'
# df = df.groupby(['Location', 'eGIFTID']).head(2).reset_index(drop=True)

# #Set Waterstops to NA
# locations = [
#     'Wellesley Finish',
#     'Bourne Finish',
#     'Foxboro/Patriot Place',
#     'Provincetown Inn',
#     'Provincetown Monuments'
# ]
# df.loc[df['Location'].isin(locations), 'Total Net Riders'] = np.nan


# filtered_df= df[df['Location'] == 'Barnstable']

# # Sort filtered_df by 'Time'
# filtered_df = filtered_df.sort_values(by='time').reset_index(drop=True)


# filtered_df

# Draft of Graph Using Plotly

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': i, 'value': i} for i in df['Location In'].unique()],
        value='Location1'  # Default value
    ),
    dcc.Graph(id='riders-plot')
])

# Define callback to update graph
@app.callback(
    Output('riders-plot', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_graph(selected_location):
    filtered_df = df[df['Location In'] == selected_location]
    fig = px.line(filtered_df, x='time', y='riders',labels={'riders': 'Number of Riders', 'Hour': 'Hour'},
                 title=f'Riders per Hour at {selected_location}')
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(mode='external')  # Use 'external' if you want to open the app in a new browser tab
